In [ ]:
local_do_arquivo = '/FileStore/cursos_prouni.csv'

In [ ]:
import pyspark.pandas as ps

df = ps.read_csv(local_do_arquivo, index_col='curso_id')

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df['curso_busca'].equals(df['nome']).sum()

In [ ]:
df = df.drop(['curso_busca'],axis=1)

In [ ]:
df.head()

In [ ]:
df = df.rename(columns={'nome':'nome_curso'})

In [ ]:
df.head()

In [ ]:
df[df['nome_curso']=='Medicina']['nome_curso'].count()

In [ ]:
df['turno'].unique()

In [ ]:
df[(df['turno']=='Integral') & (df['nome_curso']=='Medicina')]['nome_curso'].count()

In [ ]:
df[(df['turno']=='Integral') & (df['nome_curso']=='Medicina') | (df['turno']=='Matutino') & (df['nome_curso']=='Medicina') ]['nome_curso'].count()

In [ ]:
local_do_arquivo = '/FileStore/tables/reajuste.csv'
df_mensalidade = ps.read_csv(local_do_arquivo)

In [ ]:
df_mensalidade.head()

In [ ]:
df_mensalidade.shape

In [ ]:
ps.set_option('compute.ops_on_diff_frames',True)

In [ ]:
df.reset_index(inplace=True)

In [ ]:
df['mensalidade'] = df['mensalidade']+df_mensalidade['reajuste']

In [ ]:
df.dropna(inplace=True)
df.head()

In [ ]:
df.describe()

In [ ]:
df_descricao = df[df['nome_curso'] == 'Medicina'].describe()
df_descricao

In [ ]:
df_descricao_bolsa = df_descricao.drop(['mensalidade','curso_id','campus_id',"nota_integral_ampla", "nota_integral_cotas", "nota_parcial_ampla", "nota_parcial_cotas"],axis=1)
df_descricao_bolsa = df_descricao_bolsa.drop(['count'],axis=0)

In [ ]:
df_descricao_bolsa.style.format('{:,.2f}').background_gradient(axis=1)

In [ ]:
df_medicina = df.query("nome_curso == 'Medicina'")
df_medicina.head()

In [ ]:
df_medicina['mensalidade'].mean()

In [ ]:
df_medicina.query("turno == 'Integral' AND cidade_busca=='Sao Paulo'")

In [ ]:
agrupado = df_medicina.query("turno == 'Integral' OR turno == 'Matutino'").groupby('uf_busca')['mensalidade'].mean()

In [ ]:
df_agrupado = ps.DataFrame(agrupado)

In [ ]:
df_agrupado

In [ ]:
df_agrupado.sort_values(by='mensalidade',inplace=True,ascending=False)

In [ ]:
df_agrupado

In [ ]:
df_agrupado = df_agrupado.reset_index()

In [ ]:
df_agrupado.plot.bar(x='uf_busca',y='mensalidade')

In [ ]:
ps.sql('''
SELECT bolsa_integral_cotas, uf_busca
FROM {DF}
WHERE nome_curso='Medicina'
LIMIT 5;
''', DF=df)

In [ ]:
cotas_ordenadas = ps.sql('''
SELECT uf_busca, SUM(bolsa_integral_cotas) AS Total_de_cotas
FROM {DF}
WHERE nome_curso='Medicina'
GROUP BY uf_busca
ORDER BY Total_de_cotas DESC
''', DF=df)

In [ ]:
cotas_ordenadas.head()

In [ ]:
cotas_ordenadas.plot.bar(x='uf_busca',y='Total_de_cotas')

In [ ]:
df_medicina['cidade_busca'].unique()

In [ ]:
nomes_cidades = {
"Sao Luis": "Capital",
"Joao Pessoa": "Município",
"Belem": "Capital",
"Itaperuna": "Município",
"Vitoria": "Município",
"Franca": "Município",
"Sao Paulo": "Capital",
"Guarapuava": "Município",
"Campo Mourao": "Município",
"Montes Claros": "Município",
"Rio Branco": "Município",
"Imperatriz": "Município",
"Vespasiano": "Município",
"Porto Velho": "Município"
}


In [ ]:
capitais = df_medicina['cidade_busca'].map(nomes_cidades)
capitais

In [ ]:
df_medicina.insert(loc=8,column='tipo_cidade',value=capitais)


In [ ]:
df_medicina.head()

In [ ]:
cotas_ordenadas = ps.sql('''
SELECT tipo_cidade, SUM(bolsa_integral_cotas) AS Total_de_cotas
FROM {DF}
GROUP BY tipo_cidade
ORDER BY Total_de_cotas DESC
''', DF=df_medicina)
cotas_ordenadas.plot.bar(x='tipo_cidade',y='Total_de_cotas')